In [ ]:
import gradio as gr
import ollama
import time 

def chat_ollama(message, history):
    
    # Extract only the last five entries from the history and format them
    last_five = history[-5:]

    formatted_history = " ".join(f"{entry['role']}: {entry['content']}" for entry in last_five)

    stream = ollama.chat(
        model='llama3.2',
        messages=[
            {'role': 'system', 'content': "You are an intelligent and helpful assistant. Answer the current question based solely on the knowledge you have and the context from the conversation history if it is relevant. Do not explicitly mention or reference the history unless explicitly asked to recall past interactions."},
            {'role': 'user', 'content': f"""Conversation History :{formatted_history} \nQuestion: {message}"""},
        ]
    )

    response = ""
    for chunk in stream['message']['content']:
        response += chunk or ''
        yield response
        time.sleep(0.009) # to make chat generate slowly 


gr.ChatInterface(fn=chat_ollama, type="messages").launch()

In [ ]:
def chat_gpt(message,history):

    messages = [{'role':'system','content':"You are a helpful AI assistant"}]

    for user_message , assistant_message in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'user','content':assistant_message})

    messages.append({'role':'user','content':message})

    stream = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
    stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        yield response